In [15]:
%load_ext autoreload
%autoreload 2

import sys
if '../working' not in sys.path: 
    sys.path.append('../working')

!sage -preparse ../working/normalizer.sage
!mv ../working/normalizer.sage.py ../working/normalizer.py

!sage -preparse ../working/self_similar.sage
!mv ../working/self_similar.sage.py ../working/self_similar.py

from normalizer import *
from self_similar import *

prepare_gap_env()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from tqdm import tqdm

In [3]:

def permute(space, repeat) -> str:
    """ Get all the words of letters from the given
    space of given length.

    Parameters
    ----------
    space   : Iterable container of str elements of len 1.
    repeat  : int, length of yielded words
    allow_same_neighbours : if False then all words that don't have same elements
                            on the neighbour positions will be returned and all
                            possible words otherwise.
                            Default False
    Yields
    -------
    str, result word
    """
    if repeat == 1:
        for el in space:
            yield [el]
    elif repeat < 1:
        yield []
    else:
        for prev in permute(space, repeat - 1):
            for el in space:
                yield prev + [el]


def all_words(space, max_len=-1):
    """ Get all possible words (infinite set) of elements from the given
    space.

    Parameters
    ----------
    space   : Iterable container of str elements of len 1.
    allow_same_neighbours : if False then all words that don't have same elements
                            on the neighbour positions will be returned and all
                            possible words otherwise.
                            Default False.
    max_len : maximum allowed length of returned words.
              Default None, which means that there is no maximum length

    Yields
    -------
    str, result word
    """
    i = 1
    while True:
        for el in permute(space, repeat=i):
            yield el
        i += 1

        if max_len != -1 and i > max_len:
            break

def build_group(group_index, deep=3):
    e = matrix(QQ, [[1, 0], [0, 1]])
    G = to_L_basis(group_index, dim=2)
    gens = G.GeneratorsOfGroup()
    gens = [matrix(QQ, el) for el in gens]
    gens = [el for el in gens if el[:2, :2] != e]

    e = matrix(QQ, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])

    alpha = {}
    snot = []
    for seq in all_words(gens, max_len=deep): 
        
        el = e 
        for el2 in seq: 
            el *= el2 
        
        g = el[:2, :2]
        t = el[:2, 2]
        if str(g) not in alpha:
            alpha[str(g)] = t
            snot.append(block_matrix([[g, t], [0, 1]]))

    P = G.PointGroup()
    P = [matrix(QQ, el) for el in P.AsList()]

    for mtx in P:
        assert str(mtx) in alpha, 'couldnt build every element of point group'

    return G, P, alpha, snot

In [28]:
matrix.identity(QQ, 3).is_one()

True

In [4]:
group_index = 5

G, P, alpha, snot = build_group(group_index)
print('Point Group:')
print(ascii_art(P))

Point Group:
[ [1 0]  [-1  0] ]
[ [0 1], [ 1  1] ]


In [5]:
matrix(QQ, [[0, 1/2], [1, 0]]).inverse()

[0 1]
[2 0]

In [6]:
res = set()
for x in [-2, -1, 0, 1, 2]: 
    for y in [-2, -1, 0, 1, 2]: 
        res.add(x**2 + x * y + y**2)
res

{0, 1, 3, 4, 7, 12}

In [7]:
def linear_part(el): 
    n = el.dimensions()[0]
    return el[:n-1, :n-1]


def translation(el): 
    n = el.dimensions()[0]
    return el[:n-1, -1]


def construct_element(linear, trans): 
    n = linear.dimensions()[0] + 1
    res = matrix(QQ, [[0 for _ in range(n)] for _ in range(n)])
    res[:n-1, :n-1] = linear 
    res[:n-1, -1] = trans 
    res[n-1, n-1] = 1
    return res 


In [8]:

def get_word(el, gens, alpha, snot, dictionary):
    n = el.dimensions()[0] - 1 
    if str(el) in dictionary: 
        return dictionary[str(el)]
    
    trans_dict = {} 
    for e in gens: 
        if linear_part(e).is_one(): 
            i = list(translation(e).numpy().flatten()).index(1)
            trans_dict[f'e{i+1}'] = e
            i += 1 

    p = linear_part(el)
    p_snot = construct_element(p, alpha[str(p)])
    
    
    t = translation(el).numpy().flatten()
    t = t - alpha[str(p)].numpy().flatten()
    res_word = dictionary[str(p_snot)]

    for i in range (n): 
        degree = t[i]
        if degree != 0: 
            res_word = f'e{i+1}^({degree})' + res_word

    return res_word
    

def wreath_recursion(action_dict, gens, alpha, snot, dictionary):

    n = max(k[1] for k in action_dict)
    
    res = {}
    permutations = {}
    for k, v in action_dict.items(): 
        el, let = k 
        if el not in res: res[el] = [None for _ in range(n)]
        if el not in permutations: permutations[el] = [None for _ in range(n)]
        res[el][let-1] = get_word(v[1], gens, alpha, snot, dictionary)
        permutations[el][let-1] = v[0]

    
    return res, permutations

In [9]:
G, P, alpha, snot = build_group(6)
gens = [matrix(QQ, el) for el in G.GeneratorsOfGroup()]

x = (gens[0] * gens[2] ** 5 * gens[3] ** (-4))
x

[-1  0 -5]
[ 0 -1  4]
[ 0  0  1]

In [10]:
def phi_inv(g): 
    return t.inverse() * g * t

def phi(g): 
    return t * g * t.inverse() 

In [19]:
var('x1 x0')
T = matrix(QQ, [[1/2, 0], [0, 1/2]])
print(T.eigenvalues())
print(T.charpoly())
print(T.inverse())
print(T.inverse().det())
t = matrix(QQ, [[0], [0]])
t = block_matrix([[T, t], [0, 1]])
print(t)

[1/2, 1/2]
x^2 - x + 1/4
[2 0]
[0 2]
4
[1/2   0|  0]
[  0 1/2|  0]
[-------+---]
[  0   0|  1]


In [20]:
group_index = 5

G, P, alpha, snot = build_group(group_index)
gens = [matrix(QQ, el) for el in G.GeneratorsOfGroup()]


action, dictionary = self_similar(group_index, t, change_basis=True, verbose=True, gen_alphabet=True)

w_el, w_p = wreath_recursion(action, gens, alpha, snot, dictionary)

print(w_el)
print(w_p)

print('Вінцева рекурсія:')
for el in w_el.keys(): 
    p = Permutation(w_p[el])
    if p.to_permutation_group_element().order() == 1: 
        p = '()'
    else: 
        p = str(p.cycle_tuples())
    
    print(f'$${el} = {p}{w_el[el]}$$'.replace("'", '').replace('[', '(').replace(']', ')').replace('(-1)', '{-1}'))

print('```')
print(dictionary)
print('```')


conjugate el:
[-1  0  0]
[ 1  1  0]
[ 0  0  1]
conj in G: True

conjugate el:
[1 0 2]
[0 1 0]
[0 0 1]
conj in G: True

conjugate el:
[1 0 0]
[0 1 2]
[0 0 1]
conj in G: True
----------------------------------------------------
Index of subgroup H: 4
Transversal:
[1 0 0]
[0 1 0]
[0 0 1]

[1 0 1]
[0 1 0]
[0 0 1]

[1 0 0]
[0 1 1]
[0 0 1]

[-1  0  1]
[ 1  1  0]
[ 0  0  1]
Names: {'[-1  0  0]\n[ 1  1  0]\n[ 0  0  1]': 'a', '[1 0 1]\n[0 1 0]\n[0 0 1]': 'b', '[1 0 0]\n[0 1 1]\n[0 0 1]': 'c'}
ad_1:
[-1  0  0]
[ 1  1  0]
[ 0  0  1]

image:
[-1  0  0]
[ 1  1  0]
[ 0  0  1]
ad_2:
[-1  0 -1]
[ 1  1  1]
[ 0  0  1]

image:
[1 0 1]
[0 1 0]
[0 0 1]
ad_3:
[-1  0  0]
[ 1  1  1]
[ 0  0  1]

image:
[-1  0  0]
[ 1  1  1]
[ 0  0  1]
ad_4:
[ 1  0 -1]
[ 0  1  1]
[ 0  0  1]

image:
[-1  0  1]
[ 1  1  0]
[ 0  0  1]
bd_1:
[1 0 1]
[0 1 0]
[0 0 1]

image:
[1 0 1]
[0 1 0]
[0 0 1]
bd_2:
[1 0 2]
[0 1 0]
[0 0 1]

image:
[1 0 1]
[0 1 0]
[0 0 1]
bd_3:
[1 0 1]
[0 1 1]
[0 0 1]

image:
[-1  0  0]
[ 1  1  1]
[ 0  0  1]
bd_4

ValueError: an element appears twice in the input

In [179]:
dictionary

{'[-1  0  0]\n[ 0 -1  0]\n[ 0  0  1]': 'a',
 '[ 0 -1  0]\n[ 1  0  0]\n[ 0  0  1]': 'b',
 '[1 0 1]\n[0 1 0]\n[0 0 1]': 'c',
 '[1 0 0]\n[0 1 1]\n[0 0 1]': 'd',
 '[ 0  1  0]\n[-1  0  0]\n[ 0  0  1]': 'b^(-1)',
 '[ 1  0 -1]\n[ 0  1  0]\n[ 0  0  1]': 'c^(-1)',
 '[ 1  0  0]\n[ 0  1 -1]\n[ 0  0  1]': 'd^(-1)',
 '[1 0 0]\n[0 1 0]\n[0 0 1]': 'e'}

In [147]:
w_el

{'a': ['a', 'a'], 'b': ['aaab', 'aaab'], 'c': ['cd', 'cd'], 'd': ['e', 'e']}

In [136]:
w_el

{'a': ['a', 'a'], 'b': ['aaab', 'aaab'], 'c': ['cd', 'cd'], 'd': ['e', 'e']}

In [137]:
w_p

{'a': [1, 2], 'b': [1, 2], 'c': [1, 2], 'd': [2, 1]}

In [145]:
Permutation(w_p['d']).cycle_tuples()

[(1, 2)]

In [51]:
dets = {}
matrices = {}

for n in tqdm(range(2, 231)): 
    for m in normalizers(n, 3): 
        mk = str(m)
        
        if mk not in matrices: 
            matrices[mk] = []
        matrices[mk].append(n)

        md = m.det()
        if md not in dets: 
            dets[md] = []
        dets[md].append(m)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [05:13<00:00,  1.37s/it]


In [58]:
for el in matrices: print(el, '\n')

[x1  0 x2]
[ 0 x3  0]
[x4  0 x0] 

[x2  0 x3]
[ 0 x4  0]
[x0  0 x1] 

[x4  0 x0]
[ 0 x1  0]
[x2  0 x3] 

[x0  0 x1]
[ 0 x2  0]
[x3  0 x4] 

[x3  0 x4]
[ 0 x0  0]
[x1  0 x2] 

[ 0  0 x1]
[x0  0  0]
[ 0 x2  0] 

[x0  0  0]
[ 0  0 x1]
[ 0 x2  0] 

[ 0 x0  0]
[x1  0  0]
[ 0  0 x2] 

[ 0  0 x1]
[ 0 x2  0]
[x0  0  0] 

[x0  0  0]
[ 0 x1  0]
[ 0  0 x2] 

[ 0 x2  0]
[ 0  0 x1]
[x0  0  0] 

[ 0  0 x2]
[x1  0  0]
[ 0 x0  0] 

[ 0 x2  0]
[ 0  0 x0]
[x1  0  0] 

[ 0 x1  0]
[ 0  0 x0]
[x2  0  0] 

[x1  0  0]
[ 0 x2  0]
[ 0  0 x0] 

[ 0 x2  0]
[x0  0  0]
[ 0  0 x1] 

[ 0 x1  0]
[x2  0  0]
[ 0  0 x0] 

[x2  0  0]
[ 0  0 x0]
[ 0 x1  0] 

[x1  0  0]
[ 0  0 x2]
[ 0 x0  0] 

[ 0  0 x0]
[ 0 x1  0]
[x2  0  0] 

[ 0 x0  0]
[ 0  0 x2]
[x1  0  0] 

[ 0  0 x2]
[ 0 x0  0]
[x1  0  0] 

[ x0  x2   0]
[ x2 -x0   0]
[  0   0  x1] 

[ x2 -x1   0]
[ x1  x2   0]
[  0   0  x0] 

[ x2  x1   0]
[ x1 -x2   0]
[  0   0  x0] 

[ x1 -x0   0]
[ x0  x1   0]
[  0   0  x2] 

[ x0 -x2   0]
[ x2  x0   0]
[  0   0  x1] 

[ 0 x0  0]

In [54]:
list(dets.keys())

[x0*x1*x3 - x2*x3*x4,
 x1*x2*x4 - x0*x3*x4,
 -x0*x1*x2 + x1*x3*x4,
 -x1*x2*x3 + x0*x2*x4,
 x0*x2*x3 - x0*x1*x4,
 x0*x1*x2,
 -x0*x1*x2,
 -x0^2*x1 - x1*x2^2,
 x0*x1^2 + x0*x2^2,
 -x0*x1^2 - x0*x2^2,
 x0^2*x2 + x1^2*x2,
 x0^2*x1 + x1*x2^2,
 -x0^2*x1,
 2*x0*x1^2,
 -x0*x1^2,
 -2*x0*x1^2,
 x0^2*x1,
 x0*x1^2,
 -2*x0^2*x1,
 2*x0^2*x1,
 -x0^2*x2 - (x0 + x1)*x1*x2,
 x0^2*x2 + (x0 + x1)*x1*x2,
 x0^2*x1 + (x0 + x2)*x1*x2,
 x0*x1^2 + x0*(x1 + x2)*x2,
 3/4*x0*x1^2,
 -3*x0*x1^2,
 3/4*x0^2*x1,
 -3*x0^2*x1,
 3*x0*x1^2,
 -3/4*x0*x1^2,
 -3/4*x0^2*x1,
 3*x0^2*x1,
 -x0^3,
 x0^3]

In [60]:
G, P, alpha, snot = build_group(6)
print(latex([matrix(QQ, el) for el in G.GeneratorsOfGroup()]))

\left[\left(\begin{array}{rrr}
-1 & 0 & 0 \\
0 & -1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
-1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 1 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 0 \\
0 & 1 & 1 \\
0 & 0 & 1
\end{array}\right)\right]


In [9]:
print(ascii_art(snot))

[ [ 0 -1| 0]  [-1  0| 0]  [ 0 -1| 0]  [-1  1| 0]  [ 0  1| 0]  [ 1  0| 0] 
[ [ 1 -1| 0]  [ 0 -1| 0]  [-1  0| 0]  [-1  0| 0]  [-1  1| 0]  [ 1 -1| 0] 
[ [-----+--]  [-----+--]  [-----+--]  [-----+--]  [-----+--]  [-----+--] 
[ [ 0  0| 1], [ 0  0| 1], [ 0  0| 1], [ 0  0| 1], [ 0  0| 1], [ 0  0| 1],

 [1 0|0]  [0 1|0]  [-1  1| 0]  [ 1 -1| 0]  [-1  0| 0]  [ 1 -1| 0] ]
 [0 1|0]  [1 0|0]  [ 0  1| 0]  [ 1  0| 0]  [-1  1| 0]  [ 0 -1| 0] ]
 [---+-]  [---+-]  [-----+--]  [-----+--]  [-----+--]  [-----+--] ]
 [0 0|1], [0 0|1], [ 0  0| 1], [ 0  0| 1], [ 0  0| 1], [ 0  0| 1] ]


# Generate investigation for a group

In [11]:
group_index = 2

disp = 'latex'

if disp == 'latex':
    _display = latex
    title = '\\newpage\n\\subsection{Group %d}' % group_index
    section = '\\subsubsection{%s}'
    pref = '$$'
else: 
    _display = ascii_art
    title = f'=========================== Group {group_index} =================================='
    section = '\n----------------%s-------------------------\n'
    pref = '\n'

def display(*args, use_pref=True): 
    if use_pref:
        return pref + str(_display(*args)) + pref
    else: 
        return str(_display(*args))

print(title)
G, P, alpha, snot = build_group(group_index)
print('Generators of group:')
print(display([matrix(QQ, el) for el in G.GeneratorsOfGroup()]))
print('SNoT')
print(display(snot))

a0, a1 = var('a0 a1')
x = matrix([[a0], [a1]])
E = matrix(QQ, [[1, 0], [0, 1]])

print(section % 'Normalizers')   

norms = normalizers(group_index, to_l_basis=True, 
                    dim=2, verbose=False, ignore_trivial=True)

print(display(norms))

print(section % 'Dilation')   

if G.IsSymmorphicSpaceGroup(): 
    print(f"Group {group_index} is a semi-direct product, therefore the dilation part is trivial and only consists of integral vectors. ")


if disp == 'latex': 
    print('\\begin{enumerate}')
    pref2 = '\\item'
else: 
    pref2 = '...'


for A_inv in norms:
    print(pref2 + ' testing inverse A (should have integral entities):')
    print(display(A_inv))
    
    A = A_inv.inverse()
    conds = []
    variables = []
    coeffs = []
    if not G.IsSymmorphicSpaceGroup(): 
        for i, g in enumerate(P): 
            conj = (A_inv * g * A).simplify_rational()
            # conj = g
            condition = A_inv * alpha[str(g)] -  alpha[str(conj)]
            sym_res = (conj - E) * x
        
            # print(pref + '\\alpha(g) = ')
            # print(display(alpha[str(g)], use_pref=False) + pref)
            # print(pref + '\\alpha\\left(\\tau\\left(' + display(g) + '\\right)\\right) = ')
            
            alpha_conj = (E - conj) * x  + A_inv() * alpha[str(g)]
            # if disp == 'latex':
            #     print(pref + '(A^{-1}, a)' + display(block_matrix([[g, alpha[str(g)]], [0, 1]]), use_pref=False) + '(A, -Aa) = ')
            #     print(display(block_matrix([[conj, alpha_conj], [0, 1]]), use_pref=False) + '=')
            #     print(display(block_matrix([[conj, alpha[str(conj)]], [0, 1]]), use_pref=False) + pref)
            # else: 
            #     print(ascii_art('\na_inv\n\n') + ascii_art(' ')
            #           + ascii_art(block_matrix([[g, alpha[str(g)]], [0, 1]])) + ascii_art('\na\n\n') + ascii_art(' ')
            #           + ascii_art('\n=\n\n') + ascii_art(' ') + ascii_art(block_matrix([[conj, alpha_conj], [0,1]])) + ascii_art(' ')
            #           + ascii_art('\n=\n\n') + ascii_art(' ') 
            #           + ascii_art(block_matrix([[conj, alpha[str(conj)]], [0, 1]])))
            #     print()
    
            T = block_matrix([[A_inv, x], [0, 1]])
            if disp == 'latex':
                print(pref + '(A^{-1}, a)' + display(block_matrix([[g, alpha[str(g)]], [0, 1]]), use_pref=False) + '(A, -Aa) = ')
                print(display(block_matrix([[conj, alpha_conj], [0, 1]]), use_pref=False) + '=')
                print(display(block_matrix([[conj, alpha[str(conj)]], [0, 1]]), use_pref=False) + pref)
            else: 
                print(ascii_art('\na_inv\n\n') + ascii_art(' ')
                      + ascii_art(snot[i]) + ascii_art('\na\n\n') + ascii_art(' ')
                      + ascii_art('\n=\n\n') + ascii_art(' ') + ascii_art(T * snot[i] * T.inverse()) + ascii_art(' ')
                      + ascii_art('\n=\n\n') + ascii_art(' ') 
                      + ascii_art(block_matrix([[snot[i][:2, :2], alpha[str(snot[i][:2, :2])]], [0, 1]])))
                print()
                
            conds.append(sym_res[0][0] - (condition[0][0] + var(f'n{i}')))
            cond_coeffs = {}
            # for variable in conds[-1].arguments(): 
                # cur_coeff = conds[-1].coefficients(variable, sparse=False)
                # cond_coeffs[variable] = cur_coeff[1]
            # coeffs.append(cond_coeffs)
            
            conds.append(sym_res[1][0] - (condition[1][0] + var(f'm{i}')))
            # cond_coeffs = {}
            # for variable in conds[-1].arguments(): 
                # cur_coeff = conds[-1].coefficients(variable, sparse=False)
                # cond_coeffs[variable] = cur_coeff[1]
            # coeffs.append(cond_coeffs)
            
            variables.append(var(f'n{i}'))
            variables.append(var(f'm{i}'))
        
        tmp = [con == 0 for con in conds]
        print('\nequations: ')
        print(display(tmp))
        print('\nanswer:')
        res = solve(tmp, *variables)
        print(display(*res))
    
    print('Simplicity')
    print(pref + 'A = \n' + display(A.simplify_rational(), use_pref=False) + pref)
    print('\neigenvalues:')
    print(display([el[0] for el in A.charpoly().roots()]))
    print('charpoly:')
    chp = A.charpoly()(SR('x'))
    chp = factor(chp)
    print(display(chp))
    print('\nindex of subgroup:')
    print(pref + '[G : H] = \n' + display(A_inv.det(), use_pref=False) + pref)

if disp == 'latex': 
    print('\\end{enumerate}')

\newpage
\subsection{Group 2}
Generators of group:
$$\left[\left(\begin{array}{rrr}
-1 & 0 & 0 \\
0 & -1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 1 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 0 \\
0 & 1 & 1 \\
0 & 0 & 1
\end{array}\right)\right]$$
SNoT
$$\left[\left(\begin{array}{rr|r}
-1 & 0 & 0 \\
0 & -1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right), \left(\begin{array}{rr|r}
1 & 0 & 0 \\
0 & 1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right)\right]$$
\subsubsection{Normalizers}
$$\left[\right]$$
\subsubsection{Dilation}
Group 2 is a semi-direct product, therefore the dilation part is trivial and only consists of integral vectors. 
\begin{enumerate}
\end{enumerate}


# Special ocasion: group 9

In [80]:
matrix(QQ, [[-1, -2], [2, 1]]).inverse()

[ 1/3  2/3]
[-2/3 -1/3]

In [12]:
A = matrix(QQ, [[0, 1/2], [1, 0]])
t = matrix(QQ, [[0], [0]])
t = block_matrix([[A, t], [0, 1]])
self_similar(1, t, dim=2, verbose=True, change_basis=True)


conjugate el:
[1 0 0]
[0 1 2]
[0 0 1]
conj in G: True

conjugate el:
[1 0 1]
[0 1 0]
[0 0 1]
conj in G: True
----------------------------------------------------
Index of subgroup H: 2
Transversal:
[1 0 0]
[0 1 0]
[0 0 1]

[1 0 0]
[0 1 1]
[0 0 1]
a_1d_1:
[1 0 1]
[0 1 0]
[0 0 1]

a_1d_2:
[1 0 1]
[0 1 1]
[0 0 1]

a_2d_1:
[1 0 0]
[0 1 1]
[0 0 1]

a_2d_2:
[1 0 0]
[0 1 2]
[0 0 1]



({('a_1', 1): (1, 'a_1a_2'),
  ('a_1', 2): (2, 'a_1a_2'),
  ('a_2', 1): (2, 'e'),
  ('a_2', 2): (1, 'e')},
 {'[1 0 1]\n[0 1 0]\n[0 0 1]': 'a_1',
  '[1 0 0]\n[0 1 1]\n[0 0 1]': 'a_2',
  '[ 1  0 -1]\n[ 0  1  0]\n[ 0  0  1]': 'a_1^(-1)',
  '[ 1  0  0]\n[ 0  1 -1]\n[ 0  0  1]': 'a_2^(-1)',
  '[1 0 0]\n[0 1 0]\n[0 0 1]': 'e'})